In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# Ratings data.
ratings = tfds.load("movielens/latest-small-ratings", split="train")
# Features of all the available movies.
# movies = tfds.load("movielens/latest-small-movies", split="train")

In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [8]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [9]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [10]:
import tensorflow_recommenders as tfrs

In [11]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [12]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01192793]], dtype=float32)>

In [13]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [14]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [15]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [16]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [17]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 1s 12ms/step - root_mean_squared_error: 2.1629 - loss: 4.2754 - regularization_loss: 0.0000e+00 - total_loss: 4.2754
Epoch 2/3
10/10 [==============================] - 0s 8ms/step - root_mean_squared_error: 1.0326 - loss: 1.0636 - regularization_loss: 0.0000e+00 - total_loss: 1.0636
Epoch 3/3
10/10 [==============================] - 0s 7ms/step - root_mean_squared_error: 1.0159 - loss: 1.0288 - regularization_loss: 0.0000e+00 - total_loss: 1.0288


In [18]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 9ms/step - root_mean_squared_error: 1.0029 - loss: 1.0099 - regularization_loss: 0.0000e+00 - total_loss: 1.0099


{'root_mean_squared_error': 1.0029168128967285,
 'loss': 1.027868390083313,
 'regularization_loss': 0,
 'total_loss': 1.027868390083313}

In [19]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.6166008]]
Speed (1994): [[3.5971534]]
Dances with Wolves (1990): [[3.568972]]


Exporting for serving
The model can be easily exported for serving:

In [ ]:
# tf.saved_model.save(model, "export")

We can now load it back and perform predictions:

In [ ]:
# loaded = tf.saved_model.load("export")

# loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()